# Central RL control notebook

In [ ]:
import sys
from pymgrid import MicrogridGenerator as mg
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from IPython.display import display,clear_output
import pandas as pd
from Storage import StorageSuite as st
from pathlib import Path
from math import e
from time import sleep
from DQNEnv import DQAgent
import pickle as pkl
import random

Time span constants

In [ ]:
HOUR = 4 # 15 Min intervals in an hour
DAY = 96 # Number of 15 Min intervals in day
MONTH = 2_924 # Number of 15 Min intervals in month
YEAR =35_088 # Number of 15 Min intervals in year (non-leap)
ZERO = 10**-5

### Generate Microgrids

In [ ]:
n_grids = 2
grid_list = []

for i in range(n_grids):
    grid_list.append(st(r'C:\Users\thesu\Desktop\Research 2022 Microgrid\Research code repository\Micro-Grid-ML-algo\data\dummy_data_v1.csv',load=np.random.randint(1E5,6E5)))
mg_env = mg.MicrogridGenerator(storage_suite_list=grid_list)
mg_env.generate_microgrid(verbose= True, interpolate= True)

In [ ]:
print(random.randint(0,1))

In [ ]:
mg0 = mg_env.microgrids[0]
mg1 = mg_env.microgrids[1]

In [ ]:
mg_env.print_mg_parameters()

In [65]:
# li_ion, flow, flywheel = mg0.ss.unpack()
# pv =                            round(mg0.pv)
# load =                          round(mg0.load)
# net_load =                      load-pv
# status =                        mg0.grid.status
# # overgen_net_load =            pv-load
# li_ion_capa_to_charge =         li_ion.capa_to_charge
# li_ion_capa_to_discharge =      li_ion.capa_to_discharge
# flow_capa_to_charge =           flow.capa_to_charge
# flow_capa_to_discharge =        flow.capa_to_discharge
# flywheel_capa_to_charge =       flywheel.capa_to_charge
# flywheel_capa_to_discharge =    flywheel.capa_to_discharge

# li_ion_charge =                 max(0,min(-net_load,li_ion_capa_to_charge ,li_ion.power))
# li_ion_discharge =              max(0,min(net_load,li_ion_capa_to_discharge,li_ion.power))

# flow_charge =                   max(0,min(-net_load,flow_capa_to_charge ,flow.power))
# flow_discharge =                max(0,min(net_load,flow_capa_to_discharge,flow.power))

# flywheel_charge=                max(0,min(-net_load,flywheel_capa_to_charge ,flywheel.power))
# flywheel_discharge =            max(0,min(net_load,flywheel_capa_to_discharge,flywheel.power))

# li_ion_soc =                 abs(li_ion.soc)
# flow_soc =                   abs(flow.soc)
# flywheel_soc =               abs(flywheel.soc)

# capa_to_genset = mg0.genset.rated_power * mg0.genset.p_max
# p_genset = max(0, min(net_load, capa_to_genset))


# # soc_list = {'li_ion_soc': li_ion_soc, 'flow_soc': flow_soc, 'flywheel_soc': flywheel_soc}
# # soc_list = sorted(soc_list, key=lambda x: (soc_list[x]))

# control_dict = {    'pv_consummed': min(pv,load),
#                     'li_charge': 0,
#                     'li_discharge': 1E5,
#                     'flow_charge': 0,
#                     'flow_discharge': 0,
#                     'flywheel_charge': 0,
#                     'flywheel_discharge': 0,
#                     'grid_import': 0,
#                     'grid_export': 0,
#                     'genset': 0
#                 }

# new_state,reward, done, = mg0.run(control_dict)
# print(new_state)
# mg0.reset()
# print(reward)


[101247.40506372617, 0, 0.0, 0.1, 0.0, 116447.33333333333, 0.1, 0.0, 116447.33333333333, 0.1, 0.0, 116447.33333333333, 1, 0.18151899975, 0.0, 0.0]
276.929770667751


In [ ]:
# control_dict = {            'pv_consummed': min(pv,load),
#                             'li_charge': 0,
#                             'li_discharge': 0,
#                             'flow_charge': 0,
#                             'flow_discharge': 0,
#                             'flywheel_charge': 0,
#                             'flywheel_discharge': 0,
#                             'grid_import': 0,
#                             'grid_export':0,
#                             'genset': 0
#                            }

def actions_agent(mg0, action, li_ion, flow, flywheel):
    pv =                            round(mg0.pv)
    load =                          round(mg0.load)
    net_load =                      load-pv
    status =                        mg0.grid.status
    # overgen_net_load =            pv-load
    li_ion_capa_to_charge =         li_ion.capa_to_charge
    li_ion_capa_to_discharge =      li_ion.capa_to_discharge
    flow_capa_to_charge =           flow.capa_to_charge
    flow_capa_to_discharge =        flow.capa_to_discharge
    flywheel_capa_to_charge =       flywheel.capa_to_charge
    flywheel_capa_to_discharge =    flywheel.capa_to_discharge

    li_ion_charge =                 max(0,min(-net_load,li_ion_capa_to_charge ,li_ion.power))
    li_ion_discharge =              max(0,min(net_load,li_ion_capa_to_discharge,li_ion.power))

    flow_charge =                   max(0,min(-net_load,flow_capa_to_charge ,flow.power))
    flow_discharge =                max(0,min(net_load,flow_capa_to_discharge,flow.power))

    flywheel_charge=                max(0,min(-net_load,flywheel_capa_to_charge ,flywheel.power))
    flywheel_discharge =            max(0,min(net_load,flywheel_capa_to_discharge,flywheel.power))
    
    li_ion_soc =                 abs(li_ion.soc)
    flow_soc =                   abs(flow.soc)
    flywheel_soc =               abs(flywheel.soc)

    capa_to_genset = mg0.genset.rated_power * mg0.genset.p_max
    p_genset = max(0, min(net_load, capa_to_genset))
    

    # soc_list = {'li_ion_soc': li_ion_soc, 'flow_soc': flow_soc, 'flywheel_soc': flywheel_soc}
    # soc_list = sorted(soc_list, key=lambda x: (soc_list[x]))
    
    if action == 0:
        # CHARGE LI-ION
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': min(net_load,li_ion_charge),
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': 0,
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': abs(net_load)*status,
                            'genset': 0
                        }
    if action == 1:
        # DISCHARGE LI-ION
        if li_ion_soc == li_ion.MIN_SOC:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': min(net_load,li_ion_discharge),
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': abs(net_load)*status,
                                'genset': p_genset
                            }
        else:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': min(net_load,li_ion_discharge),
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': abs(net_load)*status,
                                'genset': 0
                            }
 ######################
    if action == 2:
        # CHARGE FLOW
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': min(net_load,flow_charge),
                            'flow_discharge': 0,
                            'flywheel_charge': 0,
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': abs(net_load)*status,
                            'genset': 0
                        }
    if action == 3:
        # DISCHARGE FLOW
        if flow_soc == flow.MIN_SOC:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': min(net_load,flow_discharge),
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': abs(net_load)*status,
                                'genset': p_genset
                            }
        else:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': min(net_load,flow_discharge),
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': abs(net_load)*status,
                                'genset': 0
                            }
 ######################
    if action == 4:
        # CHARGE FLYWHEEL
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': min(net_load,flywheel_charge),
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': abs(net_load)*status,
                            'genset': 0
                        }
    if action == 5:
        # DISCHARGE FLYWHEEL
        if flywheel_soc == flywheel.MIN_SOC:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': min(net_load,flywheel_discharge),
                                'grid_import': 0,
                                'grid_export': abs(net_load)*status,
                                'genset': p_genset
                            }
        else:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': min(net_load,flywheel_discharge),
                                'grid_import': 0,
                                'grid_export': abs(net_load)*status,
                                'genset': 0
                            }
 ######################
    if action == 6:
        # IMPORT
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': 0,
                            'flywheel_discharge': 0,
                            'grid_import': abs(net_load)*status,
                            'grid_export': 0,
                            'genset': 0
                        }
                        
    if action == 7:
        # EXPORT
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': 0,
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': abs(net_load)*status,
                            'genset': 0
                        }

 ######################
    if action == 8:
        # COMBINED CHARGE IMPORT
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': min(net_load/3,li_ion_charge),
                            'li_discharge': 0,
                            'flow_charge': min(net_load/3,flow_charge),
                            'flow_discharge': 0,
                            'flywheel_charge': min(net_load/3,flywheel_charge),
                            'flywheel_discharge': 0,
                            'grid_import': abs(net_load)*status,
                            'grid_export': 0,
                            'genset': 0
                        }
                        
    if action == 9:
        # COMBINED DISCHARGE EXPORT
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': 0,
                            'li_discharge': min(net_load/3,li_ion_discharge),
                            'flow_charge': 0,
                            'flow_discharge': min(net_load/3.,flow_discharge),
                            'flywheel_charge': 0,
                            'flywheel_discharge': min(net_load/3,flywheel_discharge),
                            'grid_import': 0,
                            'grid_export': abs(net_load)*status,
                            'genset': 0
                        }
        
        
    return control_dict


   

In [ ]:
# def change_name_action(idx):
# # Grid is up
#     if idx == 0:
#         action_name = "NON-ISLANDED OVERGENERATION EXPORT"
#     elif idx == 1:
#         action_name = "NON-ISLANDED OVERGENERATION EXCLUSIVE CHARGE AND EXPORT"
#     elif idx == 2:
#         action_name = "ISLANDED OVERGENERATION EXCLUSIVE CHARGE"
#     elif idx == 3:
#         action_name = "NON-ISLANDED OVERGENERATION COMBINED CHARGE"
#     elif idx == 4:
#         action_name = "ISLANDED OVERGENERATION COMBINED CHARGE"
#     elif idx == 5:
#         action_name = "NON-ISLANDED OVERGENERATION CONDITIONAL CHARGE"
#     elif idx == 6 :
#         action_name = "ISLANDED OVERGENERATION CONDITIONAL CHARGE"
#     elif idx == 7 :
#         action_name = "NON-ISLANDED UNDERGENERATION EXCLUSIVE CHARGE"
#     elif idx == 8 :
#         action_name = "ISLANDED UNDERGENERATION EXCLUSIVE DISCHARGE"
#     elif idx == 9 :
#         action_name = "NON-ISLANDED UNDERGENERATION COMBINED DISCHARGE"
#     elif idx == 10 :
#         action_name = "ISLANDED UNDERGENERATION COMBINED DISCHARGE"
#     return action_name

Now I define my agent function

In [ ]:
def training_Q_Learning(mg=mg_env.microgrids[0], n_episodes = 1, nb_actions = 8, horizon = DAY):
    '''Used to train the deep Q-learning model'''
    env = mg
    env.set_horizon(horizon = horizon) # Sets the Horizon
    # MUST DEFINE ACTION SPACE WITHIN PYMGRID
    # making an instance of the agent
    
    agent = DQAgent(learning_rate=0.05, gamma=0.90, batch_size=32, 
                     state_len=len(env.reset()), 
                     n_actions = nb_actions,
                     mem_size = 1000000,
                     min_memory_for_training=1000, epsilon=1, epsilon_dec=0.99,
                     epsilon_min = 0.02)
    
    # with open(r'C:\Users\thesu\Desktop\Trained Agent 7_2_2022.pkl', 'rb') as f:  
    #       loaded_agent = pkl.load(f)

    # agent = loaded_agent
    li_ion,flow,flywheel = mg.ss.unpack()    

    #main training loop
    for episode in range(n_episodes):
        state = env.reset()                              
        score = 0                                                                   
        done = 0
        # env._tracking_timestep = env._tracking_timestep+(HOUR*10)  
        # action_counter = np.zeros(nb_actions)                                                                     
        while not done:                                     
            # env.render()                                                         
            action_select = agent.choose_action(state)
            action = actions_agent(mg0 = env, action = action_select, li_ion = li_ion, flow = flow, flywheel = flywheel) 
            new_state,reward, done, = env.run(action)                             
            score+=reward                                                           
            agent.store_transition(state, action_select, reward, new_state, done)         
            agent.learn()                                                           
            state = new_state                                                       
            # cost_record = np.append(cost_record, abs(reward)+cost_record[i-1])
            # print(action_select)
            # print(f'Score is: {score}, Progress = {env._tracking_timestep}/{env.horizon}') 
            value_print=f"\rEpisode: {episode} Progress " + str(round(((env._tracking_timestep)*100)/(env.horizon),2))
            # action_counter[action_select] += 1
            # value_print="\r"+f"0: {action_counter[0]}, 1: {action_counter[1]}, 2: {action_counter[2]}, 3: {action_counter[3]}, 4: {action_counter[4]}, 5: {action_counter[5]}, 6: {action_counter[6]}, 7: {action_counter[7]}"
            sys.stdout.write(value_print)
            sys.stdout.flush()
    env.reset()                                                                    
    return agent

In [ ]:
capa_to_genset = mg1.genset.rated_power * mg1.genset.p_max
print(capa_to_genset)

In [ ]:
def testing_Q_Learning(mg=mg_env.microgrids[0], nb_actions = 8, horizon = DAY):
    ''' Manages the grid based on a trained neural net model '''
    env = mg
    env.set_horizon(horizon = horizon) # Sets the Horizon
    # MUST DEFINE ACTION SPACE WITHIN PYMGRID
    # making an instance of the agent
    with open(r'C:\Users\thesu\Desktop\Updated Genset Agent 7_3_2022.pkl', 'rb') as f:  
          agent = pkl.load(f)
    li_ion,flow,flywheel = mg.ss.unpack()
    state = env.reset()                                                       
    score = 0                                                                   
    done = 0
    #main testing loop                                                            
    while not done:                                                                                         
        action_select = agent.choose_action(state)
        action = actions_agent(mg0 = env, action = action_select, li_ion = li_ion, flow = flow, flywheel = flywheel) 
        new_state,reward, done, = env.run(action)                             
        score+=reward                                                                                                                   
        state = new_state
        value_print="\rProgress " + str(round(((env._tracking_timestep)*100)/(env.horizon),2)) +" %"
        sys.stdout.write(value_print)
        sys.stdout.flush()                                                       
    env.reset()                                                                    
    return score

In [29]:
def randomize(mg=mg_env.microgrids[0], nb_actions = 8, horizon = DAY):
    ''' Manages the grid with random actions. Used to compare to deep Q-learning algorithm '''
    env = mg
    env.set_horizon(horizon = horizon) # Sets the Horizon
    li_ion,flow,flywheel = mg.ss.unpack()
    state = env.reset()                                                       
    score = 0                                                                   
    done = 0
    while not done:                                                                                         
        action_select = np.random.randint(0,8)
        action = actions_agent(mg0 = env, action = action_select, li_ion = li_ion, flow = flow, flywheel = flywheel) 
        new_state,reward, done, = env.run(action)                          
        score+=reward                                                                                                                   
        state = new_state                                                       
    env.reset()
    return score 

# SCORE IS NEGATIVE, THE HIGHER THE NUMBER, THE BETTER THE PERFORMANCE

In [ ]:
trained_agent = training_Q_Learning(mg=mg1, n_episodes=1, nb_actions=8, horizon = YEAR)

In [ ]:
with open(r'C:\Users\thesu\Desktop\Updated Genset Agent 7_3_2022.pkl', 'wb') as f:  
    pkl.dump(obj =trained_agent, file=f)

Trained Network Storage

with open(r'C:\Users\thesu\Desktop\Trained Agent 7_3_2022.pkl', 'wb') as f:  
    pkl.dump(obj =trained_agent, file=f)

Object Loading

with open(r'C:\Users\thesu\Desktop\Trained Agent 7_3_2022.pkl', 'rb') as f:  
    loaded_agent = pkl.load(f)

In [ ]:
score = testing_Q_Learning(mg = mg0, nb_actions=8, horizon= YEAR)

In [31]:
r_score = randomize(mg=mg1, nb_actions = 8, horizon = YEAR)
print(r_score)

12322477.938017648


In [32]:
print(score)

20699339.289496623


In [33]:
percentage = ((score/r_score)-1)*100

In [34]:
print(str(round(percentage,2))+" %")

67.98 %
